In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import mysql
import configparser

In [2]:
content = requests.get('https://www.biat.tn/biat/Fr/cours-de-change_66_127').text 

In [3]:
soup= BeautifulSoup(content,'lxml')

In [4]:
table_content = soup.find('div', class_='table_conv')

In [5]:
data ={}
theaders = table_content.find_all('th')
for header in theaders:
    data[header.text]=[]
headers= list(data.keys())
print("The table headers are: ", headers)

trows = table_content.find_all('tr')
for row in trows: 
        row_data_list = row.find_all('td')
        for index in range(len(row_data_list)):
             data[headers[index]].append(row_data_list[index].text)
print(data)


The table headers are:  ['Désignation', 'Code', 'Unité', 'Achat', 'Vente']
{'Désignation': ['Ryal Saoudien', 'Dollar Canadien', 'Couronne Danoise', 'Dirham Emirats Arabe Unis', 'Dollar des Etats-Unis', 'Livre Sterling(2)', 'Yen Japonais', 'Dinar Koweitien', 'Couronne Norvegienne', 'Ryal Quatari', 'Couronne Suedoise', 'Franc Suisse', 'Euro', 'Dinar Bahraini', 'Yuan CHINOIS'], 'Code': ['SAR', 'CAD', 'DKK', 'AED', 'USD', 'GBP', 'JPY', 'KWD', 'NOK', 'QAR', 'SEK', 'CHF', 'EUR', 'BHD', 'CNY'], 'Unité': ['10', '1', '100', '10', '1', '1', '1000', '1', '100', '10', '10', '10', '1', '1', '1'], 'Achat': ['8.468', '2.358', '42.007', '8.587', '3.185', '3.579', '21.978', '10.199', '30.44', '8.599', '2.872', '32.639', '3.153', '8.365', '.447'], 'Vente': ['8.813', '2.456', '43.731', '8.94', '3.289', '3.727', '22.88', '10.618', '31.69', '8.953', '2.991', '33.979', '3.257', '8.709', '.466']}


In [6]:
df = pd.DataFrame(data)
print(df)

                  Désignation Code Unité   Achat   Vente
0               Ryal Saoudien  SAR    10   8.468   8.813
1             Dollar Canadien  CAD     1   2.358   2.456
2            Couronne Danoise  DKK   100  42.007  43.731
3   Dirham Emirats Arabe Unis  AED    10   8.587    8.94
4       Dollar des Etats-Unis  USD     1   3.185   3.289
5           Livre Sterling(2)  GBP     1   3.579   3.727
6                Yen Japonais  JPY  1000  21.978   22.88
7             Dinar Koweitien  KWD     1  10.199  10.618
8        Couronne Norvegienne  NOK   100   30.44   31.69
9                Ryal Quatari  QAR    10   8.599   8.953
10          Couronne Suedoise  SEK    10   2.872   2.991
11               Franc Suisse  CHF    10  32.639  33.979
12                       Euro  EUR     1   3.153   3.257
13             Dinar Bahraini  BHD     1   8.365   8.709
14               Yuan CHINOIS  CNY     1    .447    .466


In [7]:
#saving the data into a csv file
#df.to_csv(r'database.csv', index = False)

In [8]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pymysql

In [10]:
#insert your data base connection info 
scrap_db = pymysql.connect(host='XXXXXXXX',user='XXXXXXXX',password='XXXXXXXX',db='XXXXXXXX')

In [11]:
cursor = scrap_db.cursor()

In [12]:
cursor.execute("DROP TABLE IF EXISTS data ")
sql = """ CREATE TABLE data (
Désignation CHAR(60),
Code CHAR(3),
Unité INT,
Achat  FLOAT, 
Vente FLOAT
)
"""
cursor.execute(sql)

0

In [13]:
for index, row in df.iterrows(): 
    query = "INSERT INTO data (Désignation, Code, Unité, Achat ,Vente)  VALUES (%s, %s, %s, %s ,%s)"
    cursor.execute(query,tuple(row.values))

scrap_db.commit()
scrap_db.close()